In [ ]:
!pip install folium==0.12.1

In [ ]:
########## 이 부분의 코드를 실행 후, 런타임->다시시작 을 해야 한글 폰트가 적용됩니다 #######


## 한글 시각화 관련 코드
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm  # 폰트 관련 용도
# 나눔 폰트 설치
!apt-get update -qq
!apt-get install fonts-nanum* -qq
# 나눔 글꼴 사용 설정
path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
# print(font_name)
plt.rc('font', family=font_name)
# 폰트 매니저 재설정
fm._rebuild()





In [ ]:
import json
import folium
import pandas as pd
import requests
from folium.features import DivIcon


## 공공데이터 API - 데이터 불러오기

In [ ]:

# 보행어린이사고다발지역정보 Rest 조회
url = 'http://apis.data.go.kr/B552061/frequentzoneChild/getRestFrequentzoneChild'

# 일반 인증키(Encoding)
my_key = 'WUODu1UyNif9cl9LO%2FgbX43ch97p1tFDM48UM8h05ivV0NOeEYaWSBWm9LNdstWaAXZWjhY1DpUu5G%2BUqg%2BUDA%3D%3D'

params ={
        'searchYearCd' : '2020', 
        'siDo' : '11', # 서울특별시 = 11
        'guGun' : '320', 
        'type' : 'json', 
        'numOfRows' : '10', 
        'pageNo' : '1' 
}


response = requests.get(f'{url}?serviceKey={my_key}', params=params)
dict(response.json())

### 구 정보 취합

In [ ]:
guGu_codes = """강남구 680 강동구 740 강북구 305 강서구 500 관악구 620 광진구 215 구로구
530 금천구 545 노원구 350 도봉구 320 동대문구 230 동작구 590 마포구 440 서대문구
410 서초구 650 성동구 200 성북구 290 송파구 710 양천구 470 영등포구 560 용산구
170 은평구 380 종로구 110 중구 140 중랑구 260"""

guGu_codes = guGu_codes.split()

gus = [x for i, x in enumerate(guGu_codes) if i%2==0]
codes = [x for i, x in enumerate(guGu_codes) if i%2==1]

gu_dict = {gu:code for gu, code in zip(gus, codes)}
gu_dict

### 불러온 데이터 > 데이터프레임 형태로 변환

In [ ]:

df_result = pd.DataFrame()



# main url
url = 'http://apis.data.go.kr/B552061/frequentzoneChild/getRestFrequentzoneChild'

# 일반 인증키(Encoding)
my_key = 'WUODu1UyNif9cl9LO%2FgbX43ch97p1tFDM48UM8h05ivV0NOeEYaWSBWm9LNdstWaAXZWjhY1DpUu5G%2BUqg%2BUDA%3D%3D'

search_year = 2019

for gu, code in gu_dict.items():
    params ={
            'searchYearCd' : f'{search_year}',  # 검색 년도
            'siDo' : '11', # 서울특별시 = 11
            'guGun' : code, # 검색 지역
            'type' : 'json', 
            'numOfRows' : '30', 
            'pageNo' : '1' 
    }


    response = requests.get(f'{url}?serviceKey={my_key}', params=params)
    result = dict(response.json())
    
    if result['resultMsg'] == 'NODATA_ERROR':  # 요청시 데이터가 없는 경우
#         print('NODATA_ERROR', code)
        pass

    else:  # 데이터가 있으면 아래 형태로 추가
        for item in result['items']['item']:
            
            df_result = df_result.append( pd.DataFrame( {'year':search_year,
                                                         'code':code, 
                                                         '장소':item['spot_nm'], 
                                                         '발생건수':item['occrrnc_cnt'],
                                                         '위도':item['la_crd'],
                                                         '경도':item['lo_crd']
                                                        }, 
                                                       index=[0]) )
    
df_result = df_result.reset_index(drop=True) # 인덱스 초기화
df_result


## Folium 시각화

In [ ]:
import os
os.getcwd()

In [ ]:
!jupyter trust ./Seoul_accident_map.ipynb

In [ ]:
# 시도 경계 정보 다운로드
!wget https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json

In [ ]:
# 시도 경계선 추출
rfile = open('./skorea-provinces-2018-geo.json', 'r', encoding='utf-8').read()  # 파일 읽기
jsonData = json.loads(rfile)  # json load

# 대한민국 전체 json 데이터에서 서울시 데이터만 추출
seoul_json = jsonData
seoul_json['features'] = [jsonData['features'][0]]  # 가장 첫번째 feature가 서울시
# seoul_json

In [ ]:
seoul_map = folium.Map(location=[37.5502, 126.982], zoom_start=11.1, tiles='cartodbpositron')


# 서울시 경계 시각화
folium.GeoJson(seoul_json, name='json_data', style_function = lambda x :{'fillColor': '#00000000',  # 경계 내부 채움 색상 -> 비우기
                                                                        #  'color': 'black'  # 경계선 색상  # 주석 해제하면 검은색으로
                                                                         }).add_to(seoul_map)


# 지도 제목 표시

map_title = '보행자-어린이 사고'  # 표시 제목

folium.map.Marker(
    [37.6952, 126.712],  # 제목 표기 위치
    icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 20pt">{map_title}</div>',
        )
    ).add_to(seoul_map)



for idx in df_result.index:

    la_ = df_result.loc[idx,'위도']
    lo_ = df_result.loc[idx,'경도']
    occr = df_result.loc[idx,'발생건수']  # 원의 크기
    popup_text = df_result.loc[idx,'장소']  
    
    # 팝업 텍스트 박스 사이즈 조정
    iframe = folium.IFrame(popup_text)
    popup = folium.Popup(iframe,
                         min_width=200,
                         max_width=200)

    
    folium.CircleMarker(
      [la_, lo_],  # 위도 경도
      radius=int(occr) * 2,  # *(숫자)는 원의 크기가 너무 작아서 키우는 배수
      color='red',  # 원 테두리 색
      fill_color='red',  # 원 내부 색
      popup=popup  # 클릭시 나타나는 팝업 텍스트
    ).add_to(seoul_map)


seoul_map

In [ ]:
seoul_map.save('test_map.html')  # 파일 저장

# 전체 과정 함수화

In [ ]:

def get_accident_data(data_name):

    df_result = pd.DataFrame()

    # API url
    if data_name=='어린이':
        url = 'http://apis.data.go.kr/B552061/frequentzoneChild/getRestFrequentzoneChild'  # 보행자 어린이
    elif data_name == '노인':
        url = 'http://apis.data.go.kr/B552061/frequentzoneOldman/getRestFrequentzoneOldman'  # 보행자 노인
    elif data_name == '무단횡단':
        url = 'http://apis.data.go.kr/B552061/jaywalking/getRestJaywalking'  # 무단횡단
    elif data_name == '이륜차':
        url = 'http://apis.data.go.kr/B552468/acdntFreqocZone/getAcdntFreqocZone'


    # 일반 인증키(Encoding)
    my_key = 'WUODu1UyNif9cl9LO%2FgbX43ch97p1tFDM48UM8h05ivV0NOeEYaWSBWm9LNdstWaAXZWjhY1DpUu5G%2BUqg%2BUDA%3D%3D'
    
    if data_name in ['어린이', '노인', '무단횡단']:  # 이 세 가지는 API가 동일하므로 하나의 코드로 수집 가능

        for search_year in range(2016,2020):

            for gu, code in gu_dict.items():
                params ={
                        'searchYearCd' : f'{search_year}',  # 검색 년도
                        'siDo' : '11', # 서울특별시 = 11
                        'guGun' : code, # 검색 지역
                        'type' : 'json', 
                        'numOfRows' : '100', 
                        'pageNo' : '1' 
                }


                response = requests.get(f'{url}?serviceKey={my_key}', params=params)
                result = dict(response.json())

                if result['resultMsg'] == 'NODATA_ERROR':  # 요청시 데이터가 없는 경우
            #         print('NODATA_ERROR', code)
                    pass

                else:  # 데이터가 있으면 아래 형태로 추가
                    for item in result['items']['item']:

                        df_result = df_result.append( pd.DataFrame( {'year':search_year,
                                                                     'code':code, 
                                                                     '장소':item['spot_nm'], 
                                                                     '발생건수':item['occrrnc_cnt'],
                                                                     '위도':item['la_crd'],
                                                                     '경도':item['lo_crd']
                                                                    }, 
                                                                   index=[0]) )



    elif data_name == '이륜차':  # 이륜차는 API가 조금 다름

        for gu, code in gu_dict.items():
            params ={
        #             'searchYearCd' : f'{search_year}',  # 검색 년도
        #             'siDo' : '11', # 서울특별시 = 11
        #             'guGun' : code, # 검색 지역
                    'signguCode':f'11{code}',
                    'dataType' : 'JSON', 
                    'numOfRows' : '100', 
                    'pageNo' : '1' 
            }


            response = requests.get(f'{url}?serviceKey={my_key}', params=params)
            result = dict(response.json())

            if result['response']['header'] == 'NODATA_ERROR':  # 요청시 데이터가 없는 경우
        #         print('NODATA_ERROR', code)
                pass

            else:  # 데이터가 있으면 아래 형태로 추가
                for item in result['response']['body']['items']['item']:

                    df_result = df_result.append( pd.DataFrame( {
                                                                 'code':code, 
                                                                 '장소':f'{item["signguNm"]} ({item["freqocZoneNm"]})', # 서울시 종로구 (카페베네)
                                                                 '발생건수':item['acdntCo'],
                                                                 '위도':item['centerY'],
                                                                 '경도':item['centerX']
                                                                }, 
                                                               index=[0]) )

                    
    df_result = df_result.reset_index(drop=True) # 인덱스 초기화
    df_result.to_csv(f'{data_name}_data.csv', index=False, encoding='utf-8-sig')
                        
    return df_result




In [ ]:

def plot_to_map(df_data, data_name, plot_color):
    
    if data_name != '이륜차':
        map_title = f'보행자-{data_name} 사고 다발구역'
    else:
        map_title = f'이륜차 사고 다발구역'

    seoul_map = folium.Map(location=[37.5502, 126.982], zoom_start=11.1, tiles='cartodbpositron')

    # 서울시 경계 시각화
    folium.GeoJson(seoul_json, name='json_data', style_function = lambda x :{'fillColor': '#00000000',  # 경계 내부 채움 색상 -> 비우기
                                                                            #  'color': 'black'  # 경계선 색상  # 주석 해제하면 검은색으로
                                                                            }).add_to(seoul_map)


    folium.map.Marker(
        [37.6982, 126.709],  # 제목 표기 위치
        icon=DivIcon(
            icon_size=(500,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 20pt">{map_title}</div>',
            )
        ).add_to(seoul_map)


    # 범례의 스타일
    lgd_txt = '<span style="color: {col};font-size: 12pt">{txt}</span>'
                      # 색상->뒤에서 지정   # 폰트 크기
    
    # feature group ( marker가 저장될 그룹 ) 설정
    fg = folium.FeatureGroup(name= lgd_txt.format( txt= data_name, col= plot_color))
                                                    # 표시 텍스트       # 그룹 색상
    for idx in df_data.index:

        la_ = df_data.loc[idx,'위도']
        lo_ = df_data.loc[idx,'경도']
        occr = df_data.loc[idx,'발생건수']  # 원의 크기
        popup_text = df_data.loc[idx,'장소']  

        # 팝업 텍스트 박스 사이즈 조정
        iframe = folium.IFrame(popup_text)
        popup = folium.Popup(iframe,
                             min_width=200,
                             max_width=200)



        # 마커 1개씩 그리기
        marker = folium.CircleMarker(
                                      [la_, lo_],  # 위도 경도
                                      radius=int(occr) ,  # *(숫자)는 원의 크기가 너무 작아서 키우는 배수
                                      color=plot_color,  # 원 테두리 색
                                      fill_color=plot_color,  # 원 내부 색
                                      weight=1,  # 원 테두리 두께
                                      popup=popup  # 클릭시 나타나는 팝업 텍스트
                                    )

        fg.add_child(marker)  # 그린 마커를 feature group에 추가
        
    seoul_map.add_child(fg)  # feature group을 map에 추가


    folium.map.LayerControl('topleft', collapsed= False).add_to(seoul_map)  # map에 범례" 추가


    
    seoul_map.save(f'{map_title}.html')  # 파일 저장 - 파일명==지도제목
    
    return seoul_map


# 개별 데이터 시각화

In [ ]:
# 수집할 데이터 이름

data_name = '어린이'  # 어린이/노인/무단횡단/이륜차 중 하나
plot_color = 'red'

df_accident = get_accident_data(data_name)  # API에서 데이터 수집 -> pandas dataframe으로 저장

plot_to_map(df_accident, data_name, plot_color)  # dataframe -> folium 시각화
# 함수 내부에서 html로 저장까지 이루어짐


In [ ]:
# 수집할 데이터 이름

data_name = '노인'  # 어린이/노인/무단횡단/이륜차 중 하나
plot_color = 'orange'  # yellow가 너무 형광색이라 변경 / 다른 색상 이름: gold, blue, skyblue 등

df_accident = get_accident_data(data_name)  # API에서 데이터 수집 -> pandas dataframe으로 저장

plot_to_map(df_accident, data_name, plot_color)  # dataframe -> folium 시각화
# 함수 내부에서 html로 저장까지 이루어짐


In [ ]:
# 수집할 데이터 이름

data_name = '무단횡단'  # 어린이/노인/무단횡단/이륜차 중 하나
plot_color = 'green'

df_accident = get_accident_data(data_name)  # API에서 데이터 수집 -> pandas dataframe으로 저장

plot_to_map(df_accident, data_name, plot_color)  # dataframe -> folium 시각화
# 함수 내부에서 html로 저장까지 이루어짐


In [ ]:
# 수집할 데이터 이름

data_name = '이륜차'  # 어린이/노인/무단횡단/이륜차 중 하나
plot_color = 'purple'

df_accident = get_accident_data(data_name)  # API에서 데이터 수집 -> pandas dataframe으로 저장

plot_to_map(df_accident, data_name, plot_color)  # dataframe -> folium 시각화
# 함수 내부에서 html로 저장까지 이루어짐


# 전체 데이터 시각화

## 데이터 수집

In [ ]:
# 4개 종류 데이터를 각각 수집

df_list = []

name_list = ['어린이', '노인', '무단횡단', '이륜차']
color_list = ['red', 'orange', 'green', 'purple']

for data_name, plot_color in zip(name_list, color_list):  # 각각의 리스트에서 (name, color)를 쌍으로 꺼내와서 아래 코드 실행

    df_accident = get_accident_data(data_name)  # API에서 데이터 수집 -> pandas dataframe으로 저장
    df_accident['color'] = plot_color  # 데이터프레임에 시각화 색상 저장
    df_accident['acdt_name'] = data_name  # 사고 이름
    
    df_list.append(df_accident)
    
df_total = pd.concat(df_list).reset_index(drop=True)

# 구 이름 정보 추가
df_total['구'] = df_total['장소'].apply(lambda x: ' '.join(x.split()[0:2]))


df_total



### 구 별 시각화

### 데이터 필터링 -> 시각화

In [ ]:

filter_acdt = '어린이'

df_filtered = df_total[df_total['acdt_name']==filter_acdt]  # 정해진 사고 목록만 가져오기

df_acdt_by_gu = df_filtered.groupby('구').sum()[['발생건수']]  # 위에서 필터링한 데이터에서 구 별 사고 총합 계산
top5_gus = df_acdt_by_gu.index[:5]  # 상위 5개 구만 선택

df_filtered = df_filtered[df_filtered['구'].str.contains('|'.join(top5_gus))]  # 상위 5개 구에 속하는 데이터만 필터링
df_filtered = df_filtered.groupby('구').sum()[['발생건수']].sort_values(by='발생건수',ascending=False).reset_index() 
                 # 필터링된 데이터의 구 별 발생건수 총합  -> 발생건수 내림차순으로 정렬 -> 인덱스 초기화

# 파일로 저장
df_filtered.to_csv(f'./{filter_acdt}_교통사고_상위5개구.csv', encoding='utf-8-sig', index=False)

df_filtered

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(figsize=(10,5), dpi=150) # figsize=그림 크기, dpi=해상도
fig.patch.set_facecolor('white')  # 그림 배경을 흰색으로
sns.barplot(x='구', y='발생건수', data=df_filtered, ax=ax)
ax.set_title(f'구 별 {filter_acdt} 사고 발생 건수 (상위 5개 구)', fontsize=20, pad=20)  # 제목 설정, 폰트 크기, 제목-그래프 여백 크기

plt.show()

In [ ]:

filter_acdt = '노인'

df_filtered = df_total[df_total['acdt_name']==filter_acdt]  # 정해진 사고 목록만 가져오기

df_acdt_by_gu = df_filtered.groupby('구').sum()[['발생건수']]  # 위에서 필터링한 데이터에서 구 별 사고 총합 계산
top5_gus = df_acdt_by_gu.index[:5]  # 상위 5개 구만 선택

df_filtered = df_filtered[df_filtered['구'].str.contains('|'.join(top5_gus))]  # 상위 5개 구에 속하는 데이터만 필터링
df_filtered = df_filtered.groupby('구').sum()[['발생건수']].sort_values(by='발생건수',ascending=False).reset_index() 
                 # 필터링된 데이터의 구 별 발생건수 총합  -> 발생건수 내림차순으로 정렬 -> 인덱스 초기화

df_filtered.to_csv(f'./{filter_acdt}_교통사고_상위5개구.csv', encoding='utf-8-sig', index=False)



fig, ax = plt.subplots(figsize=(10,5), dpi=150)
fig.patch.set_facecolor('white')
sns.barplot(x='구', y='발생건수', data=df_filtered, ax=ax)
ax.set_title(f'구 별 {filter_acdt} 사고 발생 건수 (상위 5개 구)', fontsize=20, pad=20)

plt.show()

- 출력된 이미지는 우클릭 -> 다른이름으로저장 으로 다운로드 가능

## 데이터 시각화

In [ ]:

# 위에서 수집한 전체 데이터를 한 번에 plot

# 맵 생성
seoul_map = folium.Map(location=[37.5502, 126.982], zoom_start=11.1, tiles='cartodbpositron')

# 서울시 경계 시각화
folium.GeoJson(seoul_json, name='json_data', style_function = lambda x :{'fillColor': '#00000000',  # 경계 내부 채움 색상 -> 비우기
                                                                        #  'color': 'black'  # 경계선 색상  # 주석 해제하면 검은색으로
                                                                        }).add_to(seoul_map)


# 지도 제목 표시
map_title = '전체 사고 다발 지역'  # 표시 제목

folium.map.Marker(
    [37.6982, 126.709],  # 제목 표기 위치
    icon=DivIcon(
        icon_size=(500,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 20pt">{map_title}</div>',
        )
    ).add_to(seoul_map)




# 범례의 스타일
lgd_txt = '<span style="color: {col};font-size: 12pt">{txt}</span>'
                  # 색상->뒤에서 지정   # 폰트 크기


df_data = df_total  # 시각화에 사용할 데이터프레임은 위에서 만든 전체 사고 기록 데이터

for acdt_name in df_total['acdt_name'].unique():  # 사고 종류에 따라 group을 지어 시각화
        
    df_data = df_total[df_total['acdt_name']==acdt_name]  # 현재 보고있는 사고 이름(어린이/노인/무단횡단/이륜차)만 아래 코드 실행
    plot_color = df_data['color'].tolist()[0]  # 현재 선택 사고의 색상 가져오기
    
    fg = folium.FeatureGroup(name= lgd_txt.format( txt= acdt_name, col= plot_color))
    # marker를 저장할 feature group 설정 - 표시 텍스트 속성: lgd_txt, 표시 텍스트: acdt_name, 표시 색상: plot_color

    for idx in df_data.index:  # > 이륜차 데이터가 많아서 먼저 그려지도록 
                                          #    순서를 뒤집어서(이륜차가 아래에 그려지도록) 실행
        la_ = df_data.loc[idx,'위도']
        lo_ = df_data.loc[idx,'경도']

        occr = df_data.loc[idx,'발생건수']  # 원의 크기
#         plot_color = df_data.loc[idx, 'color']  # 원 색상  # > 위에서 지정
#         acdt_name = df_data.loc[idx, 'acdt_name']          # > 위에서 지정

        popup_text = df_data.loc[idx,'장소']  

        # 팝업 텍스트 박스 사이즈 조정
        iframe = folium.IFrame(popup_text)
        popup = folium.Popup(iframe,
                             min_width=200,
                             max_width=200)


        # 마커 1개씩 그리기
        marker = folium.CircleMarker(
                                      [la_, lo_],  # 위도 경도
                                      radius=int(occr) ,  # *(숫자)는 원의 크기가 너무 작아서 키우는 배수
                                      color=plot_color,  # 원 테두리 색
                                      fill_color=plot_color,  # 원 내부 색
                                      weight=1,  # 원 테두리 두께
                                      popup=popup  # 클릭시 나타나는 팝업 텍스트
                                    )

        fg.add_child(marker)  # 그린 마커를 feature group에 추가
        
    seoul_map.add_child(fg)  # feature group을 map에 추가



folium.map.LayerControl('topleft', collapsed= False).add_to(seoul_map)  # map에 범례" 추가



seoul_map.save(f'{map_title}.html')  # 파일 저장 - 파일명==지도제목

seoul_map


- 왼쪽 상단 범례에서 체크하는 걸로 데이터 시각화를 보고 끌 수 있음
    - 껐다 켜면 기존 시각화 "위에" 새로 그려짐
    - 기본 출력에서 `무단횡단->노인->어린이` 순으로 껐다 켜시면 시각화가 더 잘 보입니다
    
- 확대해서 보면 사고 지역 특징이 더 잘 나옵니다
  - 이륜차 사고는 대부분 사거리에서 일어나는 듯 합니다
  - 이륜차 사고를 제외하고, 3가지 보행자 사고가 몰아서 나는 곳도 있습니다(신림동 좌측)
  - 노인 사고는 동대문구와 종로구에서 집중적으로 많이 일어납니다
  